https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series

In [ ]:
data = pd.read_csv(r"D:/data_files/train_u6lujuX_CVtuZ9i.csv",index_col = "Loan_ID",engine='python')
data.head()

# Boolean Indexing

In [ ]:
data.loc[(data['Gender']=='Female')&(data['Education']=='Not Graduate') & (data['Loan_Status']=="Y"),["Gender","Education","Loan_Status"]]

# Apply Function

In [ ]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
print(data.apply(num_missing,axis=0))   #axis=0 defines that function is to be applied on each column

#Applying per row:
print("Missing values per row:")
print(data.apply(num_missing,axis=1).head()) #axis=1 defines that function is to be applied on each row

# Imputing missing files

‘fillna()’ does it in one go. It is used for updating missing values with the overall mean/mode/median of the column. Let’s impute the ‘Gender’, ‘Married’ and ‘Self_Employed’ columns with their respective modes.

In [ ]:
#first we import a function to determine the mode
from scipy.stats import mode
print(mode(data['Gender'].dropna()))
print(mode(data['Gender'].dropna()).mode[0])

注意：原文中，mode(data['Gender'])会报错，指出scipy的mode函数无法处理列表中包含混合类型的情况，比方说上面的例子就是包含了缺失值NAN类型和字符串类型，所以无法直接处理。

同时也指出Pandas自带的mode函数是可以处理混合类型的：

In [ ]:
from pandas import Series
Series.mode(data['Gender'])[0]

In [ ]:
#Impute the values:
data['Gender'].fillna(Series.mode(data['Gender'])[0],inplace=True)
data['Married'].fillna(Series.mode(data['Married'])[0],inplace=True)
data['Self_Employed'].fillna(Series.mode(data['Self_Employed'])[0],inplace=True)

#Now check the missing values again to confirm:
print(data.apply(num_missing,axis=0))

# Pivot Table

In [ ]:
#Determine pivot table
impute_grps = data.pivot_table(values=["LoanAmount"],index=["Gender","Married","Self_Employed"],aggfunc=np.mean)
print(impute_grps)

# Multi-Indexing

In [ ]:
#iterate only through rows with missing LoanAmount
for i,row in data.loc[data['LoanAmount'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Married'],row['Self_Employed']])
    data.loc[i,'LoanAmount'] = impute_grps.loc[ind].values[0]
    
#Now check the missing values again to confirm:
print(data.apply(num_missing,axis=0))

# Crosstab

This function is used to get an initial “feel” (view) of the data. Here, we can validate some basic hypothesis. For instance, in this case, “Credit_History” is expected to affect the loan status significantly. This can be tested using cross-tabulation as shown below:

In [ ]:
pd.crosstab(data["Credit_History"],data['Loan_Status'],margins=True)

These are absolute numbers. But, percentages can be more intuitive in making some quick insights. We can do this using the apply function:

In [ ]:
def perConvert(ser):
    return ser/float(ser[-1])

pd.crosstab(data['Credit_History'],data['Loan_Status'],margins=True).apply(perConvert,axis=1)

# Merge DataFrames

Merging dataframes become essential when we have information coming from different sources to be collated. Consider a hypothetical case where the average property rates (INR per sq meters) is available for different property types. Let’s define a dataframe as:

In [ ]:
prop_rates = pd.DataFrame([1000, 5000, 12000], index=['Rural','Semiurban','Urban'],columns=['rates'])
prop_rates

In [ ]:
data_merged = data.merge(right=prop_rates,how='inner',left_on='Property_Area',right_index=True,sort=False)
data_merged.pivot_table(values='Credit_History',index=['Property_Area','rates'],aggfunc=len)

# Sorting DataFrames

In [ ]:
data_sorted = data.sort_values(['ApplicantIncome','CoapplicantIncome'], ascending=False)
data_sorted[['ApplicantIncome','CoapplicantIncome']].head(10)

# Plotting (Boxplot & Histogram)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
data.boxplot(column="ApplicantIncome",by="Loan_Status")

In [ ]:
data.hist(column="ApplicantIncome",by="Loan_Status",bins=30)

# Cut function for binning

Sometimes numerical values make more sense if clustered together. For example, if we’re trying to model traffic (#cars on road) with time of the day (minutes). The exact minute of an hour might not be that relevant for predicting traffic as compared to actual period of the day like “Morning”, “Afternoon”, “Evening”, “Night”, “Late Night”. Modeling traffic this way will be more intuitive and will avoid overfitting.

Here we define a simple function which can be re-used for binning any variable fairly easily.

In [ ]:
#Binning:
def binning(col,cut_points,labels=None):
    #Define min and max values:
    minval = col.min()
    maxval = col.max()
    
    #create list by adding min and max to cut_points
    break_points = [minval] + cut_points + [maxval]
    
    #if no labels provided, use default labels 0...n-1
    if not labels:
        labels = range(len(cut_points)+1)
    #Binning using cut function of pandas
    colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
    return colBin

#Bining age:
cut_points=[90,140,190]
labels =['low','medium','high','very high']
data['LoanAmount_Bin'] = binning(data['LoanAmount'],cut_points,labels)
print(pd.value_counts(data['LoanAmount_Bin'],sort=False))

# Coding nominal data

Often, we find a case where we’ve to modify the categories of a nominal variable. This can be due to various reasons:

1.Some algorithms (like Logistic Regression) require all inputs to be numeric. So nominal variables are mostly coded as 0, 1….(n-1)

2.Sometimes a category might be represented in 2 ways. For e.g. temperature might be recorded as “High”, “Medium”, “Low”, “H”, “low”. Here, both “High” and “H” refer to same category. Similarly, in “Low” and “low” there is only a difference of case. But, python would read them as different levels.

3.Some categories might have very low frequencies and its generally a good idea to combine them.
Here I’ve defined a generic function which takes in input as a dictionary and codes the values using ‘replace’ function in Pandas.

In [ ]:
#Define a generic function using Pandas replace function
def coding(col,codeDict):
    colCoded = pd.Series(col,copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key,value,inplace=True)
    return colCoded

#Coding LoanStatus as Y=1,N=0:
print('Before Coindg:')
print(pd.value_counts(data['Loan_Status']))
data['Loan_Status_Coded'] = coding(data['Loan_Status'],{'N':0,'Y':1})
print('\nAfter Coding:')
print(pd.value_counts(data['Loan_Status_Coded']))

# Iterating over rows of a dataframe

This is not a frequently used operation. Still, you don’t want to get stuck. Right? At times you may need to iterate through all rows using a for loop. For instance, one common problem we face is the incorrect treatment of variables in Python. This generally happens when:

1.Nominal variables with numeric categories are treated as numerical.
2.Numeric variables with characters entered in one of the rows (due to a data error) are considered categorical.

So it’s generally a good idea to manually define the column types. If we check the data types of all columns:

In [ ]:
data.dtypes

Here we see that Credit_History is a nominal variable but appearing as float. A good way to tackle such issues is to create a csv file with column names and types. This way, we can make a generic function to read the file and assign column data types. For instance, here I have created a csv file 

In [ ]:
colTypes = pd.read_csv(r'D:/data_files/datatypes.csv',engine='python')
print(colTypes)

After loading this file, we can iterate through each row and assign the datatype using column ‘type’ to the variable name defined in the ‘feature’ column.

In [ ]:
data = data.reset_index()

In [ ]:
#Iterate through each row and assign variable type.
#Note: astype is used to assign types

for i, row in colTypes.iterrows():  #i: dataframe index; row: each row in series format
    if row['type']=="categorical":
        data[row['feature']]=data[row['feature']].astype(np.object)
    elif row['type']=="continuous":
        data[row['feature']]=data[row['feature']].astype(np.float)
print (data.dtypes)   